In [192]:
import os, re, io
import numpy as np
import pandas as pd


In [193]:

# from database_setup import engine
from sqlalchemy.orm import sessionmaker
from models import BranchProduct, Product
from database_setup import engine


In [194]:
PROJECT_DIR = '/home/rafaelleonardo/Python_projects/jupiter_scrapy'
ASSETS_DIR = os.path.join(PROJECT_DIR, "assets")
PRODUCTS_PATH = os.path.join(ASSETS_DIR, "PRODUCTS.csv")
PRICES_STOCK_PATH = os.path.join(ASSETS_DIR, "PRICES-STOCK.csv")

In [195]:
products_df = pd.read_csv(filepath_or_buffer=PRODUCTS_PATH, sep="|",)
prices_stock_df = pd.read_csv(filepath_or_buffer=PRICES_STOCK_PATH, sep="|",)

In [201]:
prices_stock_df = prices_stock_df[(prices_stock_df.BRANCH == 'MM') | (prices_stock_df.BRANCH == 'RHSM')]
products_df['DESCRIPTION'] = products_df['DESCRIPTION'].str.replace('<[^<]+?>', '')
products_df['CATEGORY'] = products_df['CATEGORY'].str.lower() + '|' + products_df['SUB_CATEGORY'].str.lower() 

In [202]:
prices_stock_df = prices_stock_df[(prices_stock_df['STOCK'] > 0)]

In [203]:
prices_stock_df.shape

(460002, 4)

In [200]:
products_df = products_df[products_df.SKU.isin(prices_stock_df.SKU.unique())]

In [175]:
products_df.shape

(306768, 14)

In [176]:
del products_df['BUY_UNIT']
del products_df['DESCRIPTION_STATUS']
del products_df['ORGANIC_ITEM']
del products_df['KIRLAND_ITEM']
del products_df['FINELINE_NUMBER']
del products_df['SUB_SUB_CATEGORY']
del products_df['SUB_CATEGORY']

In [159]:
products_df.rename(columns={
    'SKU': 'sku',
    'BARCODES': 'barcodes',
    'NAME': 'name',
    'DESCRIPTION': 'description',
    'IMAGE_URL': 'image_url',
    'CATEGORY': 'category',
    'BRAND': 'brand',
}, inplace=True)

prices_stock_df.rename(columns={
    'SKU': 'sku',
    'BRANCH': 'branch',
    'PRICE': 'price',
    'STOCK': 'stock',
}, inplace=True)

In [162]:
prices_stock_df.head()

,sku,branch,price,stock
1,223240,RHSM,15.3,5
2,49514,RHSM,56.7,-2
3,111807,RHSM,48.0,-4
13,22797,RHSM,32.4,-2
15,85778,RHSM,64.2,5


In [51]:
def get_package(row):
    temp_list = row['DESCRIPTION'].split()
    if len(temp_list) == 1:
        return np.nan
    word1 = temp_list[-1]
    word2 = temp_list[-2]
    try: 
        int(word1)
        return np.nan
    except:
        if bool(re.search(r'\d', word1)):
            return word1
        else:
            try:
                int(word2)
                return word2 + ' ' + word1
            except:
                return np.nan

In [52]:
def extract_description(row):
    temp_list = row['DESCRIPTION'].split()
    if len(temp_list) == 1:
        return row['DESCRIPTION']
    word1 = temp_list[-1]
    word2 = temp_list[-2]
    try: 
        int(word1)
        return row['DESCRIPTION']
    except:
        if bool(re.search(r'\d', word1)):
            return row['DESCRIPTION'].replace(' ' + word1, '')
        else:
            try:
                int(word2)
                return row['DESCRIPTION'].replace(' ' + word2 + ' ' + word1, '')
            except:
                return row['DESCRIPTION']   

In [19]:
def get_package(row):
    temp_list = row['DESCRIPTION'].split()
    if len(temp_list) == 1:
        return np.nan
    word1 = temp_list[-1]
    word2 = temp_list[-2]
    try: 
        int(word1)
        return np.nan
    except:
        if bool(re.search(r'\d', word1)):
            return word1
        else:
            try:
                int(word2)
                return word2 + ' ' + word1
            except:
                return np.nan
            

In [141]:
products_df['store'] = 'Richart Wholesale Wlub'

In [145]:
products_df

,SKU,BUY_UNIT,DESCRIPTION_STATUS,ORGANIC_ITEM,KIRLAND_ITEM,FINELINE_NUMBER,BARCODES,NAME,DESCRIPTION,IMAGE_URL,CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY,BRAND,store
0,52572,UN,B,NaN,False,713115,693177767936,CANASTOS,CANASTO CONEJO F1 A 1UN,https://locahost:8000/images/693177767936?heig...,apparel|mens wear,MENS WEAR,MEN - T-SHIRTS,PUPPY DOG PALS,Richart Wholesale Wlub
1,278100,NaN,R,NaN,True,345827,399,LIMÓN COLIMA KG,LIMÓN COLIMA KG,https://locahost:8000/images/399?height=500&wi...,entertainment|toys & hobbies,TOYS & HOBBIES,PELUCHES,ENTRECOT,Richart Wholesale Wlub
2,27404,KG,B,NaN,True,134923,762230099043,CHICLES,TRIDENT 6S SANDIA 9GR,https://locahost:8000/images/762230099043?heig...,hardlines|automotive,AUTOMOTIVE,APARIENCIA AUTOMOVIL,SPURA,Richart Wholesale Wlub
3,215143,UN,CD,Y,True,773663,7501199416236,JABON LIQUIDO PARA MANOS VITAMINA E,JABON LIQUIDO PARA MANOS VITAMINA E 442 ML.,https://locahost:8000/images/7501199416236?hei...,hardlines|seasonal,SEASONAL,SEASONAL - EASTER,CHAMYTO,Richart Wholesale Wlub
4,85805,KG,CD,N,False,233313,8410113003119,VINO TINTO GRAN CORONAS TORRES 750,VINO TINTO GRAN CORONAS TORRES 750 1 PZA,https://locahost:8000/images/8410113003119?hei...,consumables|household chemicals,HOUSEHOLD CHEMICALS,AIR FRESHENER-DEODORIZER,POOPSIE,Richart Wholesale Wlub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321537,37542,UN,NaN,N,False,966232,698400300544,LOZA BASICA GRA,TAZA CONSOME 280CC 1UN,https://locahost:8000/images/698400300544?heig...,hardlines|hardware,HARDWARE,VENTILACION,HO DON CUSTODIO,Richart Wholesale Wlub
321538,199851,NaN,R,Y,True,855034,7501199404301,DETERGENTE LIQUIDO COLOR,DETERGENTE LIQUIDO COLOR 5 LT,https://locahost:8000/images/7501199404301?hei...,grocery|dairy products,DAIRY PRODUCTS,BUTTER-SUBSTITUTES,MADEMSA,Richart Wholesale Wlub
321539,144269,NaN,R,Y,True,434058,7501006555752,2 SOBRES PALOMITAS + VALENTINA,2 SOBRES PALOMITAS + VALENTINA 1 PZA,https://locahost:8000/images/7501006555752?hei...,entertainment|electronics,ELECTRONICS,EQUIPOS DE AUDIO,DON JUAN,Richart Wholesale Wlub
321540,312844,KG,B,NaN,True,305744,6971165068775,"CAMISA JORDACHE J, BLANCO, EG/XL","CAMISA JORDACHE J, BLANCO, EG/XL",https://locahost:8000/images/6971165068775?hei...,grocery|92 grocery,92 GROCERY,MILK-CREAM-SUBSTITUTES-SHELF,MEYER,Richart Wholesale Wlub


In [205]:
products_df['NAME'].notna()

0         True
1         True
2         True
3         True
4         True
          ... 
321537    True
321538    True
321539    True
321540    True
321541    True
Name: NAME, Length: 315119, dtype: bool

In [206]:
products_df['NAME']

0                                    CANASTOS
1                          LIMÓN COLIMA KG   
2                                     CHICLES
3         JABON LIQUIDO PARA MANOS VITAMINA E
4          VINO TINTO GRAN CORONAS TORRES 750
                         ...                 
321537                        LOZA BASICA GRA
321538               DETERGENTE LIQUIDO COLOR
321539        2 SOBRES PALOMITAS  + VALENTINA
321540       CAMISA JORDACHE J, BLANCO, EG/XL
321541                                   SOFA
Name: NAME, Length: 315119, dtype: object

(306768, 14)

(306768, 13)

(14774, 13)

(321542, 13)

321542